In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import pandas as pd
import random
import scipy

from scipy.stats import zscore

# interactive
from ipywidgets.widgets import interact, IntSlider, FloatSlider
from IPython.display import display


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from multiDatenanalyse import *

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

mmPfad = '../data/Messmatrix.csv'

# Load data

In [ ]:
df = load_data()

# Params

# Extract data according to product

In [ ]:
min_num_walzlos = 300
df_all_prod = [extract_product(df, product_id=product_id, min_num_walzlos=min_num_walzlos) for product_id in range(26)]

In [ ]:
@interact(index=IntSlider(min=0, max=26, value = 0))
def count_per_product(index):
    print("Anzahl der Walzlose: "+str(len(pd.unique(df_all_prod[index]["Header_Walzlos"]))))

In [ ]:
product_id = 21
df_prod = df_all_prod[product_id]
print("Anzahl der Walzlose: "+str(len(pd.unique(df_prod["Header_Walzlos"]))))

In [ ]:
df_prod.columns

# Rearrange data for lda

In [ ]:
test_frac = 0.4
train_set, test_set = get_lda_data(df_prod, test_frac=test_frac)

# Normalize data

In [ ]:
train_set['data'] = zscore(train_set['data'])
test_set['data'] = zscore(test_set['data'])

# How different are train and test set

In [ ]:
cov_train = np.cov(train_set['data'].T)
cov_test = np.cov(test_set['data'].T)

plt.figure(figsize=(15,10))
ax1 = plt.subplot(121)
ax1.imshow(255*(cov_train-np.max(cov_train))/(np.max(cov_train)-np.min(cov_train)), 'gray')
ax1.set_title('cov train')
ax1.set_xlabel('features')
ax1.set_ylabel('features')

ax2 = plt.subplot(122)
ax2.imshow(255*(cov_test-np.max(cov_test))/(np.max(cov_test)-np.min(cov_test)), 'gray')
ax2.set_title('cov test')
ax2.set_xlabel('features')
ax2.set_ylabel('features')
print('Wie ähnlich sind sich test und train set?')

# LDA

In [ ]:
# extract data and label
X_train, y_train = train_set['data'], train_set['label']
X_test, y_test = test_set['data'], test_set['label']

# number components for transofrm
n_components = 3

# LDA object
sklearn_LDA = LDA(n_components=n_components, solver='eigen')

# fit with train data
sklearn_LDA = sklearn_LDA.fit(X_train, y_train)

# Explained Variance Ratio

In [ ]:
plt.stem(sklearn_LDA.explained_variance_ratio_)
plt.xlabel('Index Eigenwert')
plt.ylabel('Beitrag zur Varianz')

# How good can LDA classify?

In [ ]:
train_pred = sklearn_LDA.predict(X_train)
print('{0:.2f}% train accuracy'.format(100*np.mean(train_pred == y_train)))

test_pred = sklearn_LDA.predict(X_test)
print('{0:.2f}% test accuracy'.format(100*np.mean(test_pred == y_test)))

# Plot LDA

In [ ]:
data = sklearn_LDA.transform(X_train)
plot_lda(data, y_train, 'First two LDA components')

# Interpret LDA

In [ ]:
eigvecs = sklearn_LDA.scalings_
plt.figure(figsize=(20,5))
plt.imshow(np.abs(eigvecs), 'gray')
_ = plt.axis('off')

In [ ]:
print('Most valuable component in most valuable EV: {}'.format(df[df.columns[6:]].columns[np.argmax(np.abs(eigvecs[:, 0]))]))
print('Most valuable component in second valuable EV: {}'.format(df[df.columns[6:]].columns[np.argmax(np.abs(eigvecs[:, 1]))]))

In [ ]:
plt.figure(figsize=(20,10))

for index in range(6):
    ax = plt.subplot(2,3,index+1)
    ax.stem(eigvecs[:, index])
    ax.set_title('Eigenvector {}'.format(index))
    ax.set_xlabel('Merkmalsindex')